# query()와 eval()의 등장배경:복합 표현식

In [1]:
import numpy as np
rng=np.random.RandomState(42)
x=rng.rand(1000000)
y=rng.rand(1000000)
%timeit x+y

6.71 ms ± 512 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
%timeit np.fromiter((xi+yi for xi, yi in zip(x,y)),dtype=x.dtype, count=len(x))

756 ms ± 17.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
mask=(x>0.5)&(y<0.5)

In [4]:
tmp1=(x>0.5)
tmp2=(y<0.5)
mask=tmp1&tmp2

In [5]:
#모든 중간 단계가 명시적으로 메모리에 할당
#Numexpr 라이브러리를 사용하면 중간 배열을 할당하지 않고도 요소별로 이러한 유형의 복합 표현식을 계산
import numexpr
mask_numexpr=numexpr.evaluate('(x>0.5) & (y<0.5)')
np.allclose(mask,mask_numexpr)

True

# 효율적인 연산을 위한 pandas.eval()

In [6]:
import pandas as pd
nrows,ncols=100000, 100
rng=np.random.RandomState(42)
df1,df2,df3,df4 = (pd.DataFrame(rng.rand(nrows,ncols))
                  for i in range(4))

In [7]:
#이 표현식의 eval()버전은 같은 결과를 주면서 50% 더 빠르고 메모리도 훨씬 더 적게 사용
%timeit pd.eval('df1+df2+df3+df4')

61 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
np.allclose(df1+df2+df3+df4,
           pd.eval('df1+df2+df3+df4'))

True

In [9]:
#pd.eval()이 지원하는 연산
df1,df2,df3,df4,df5=(pd.DataFrame(rng.randint(0,1000,(100,3)))
                    for i in range(5))

In [10]:
#산술 연산자 pd.eval()은 모든 산술 연산자를 지원
result1=-df1*df2 / (df3+df4)-df5
result2=pd.eval('-df1*df2 / (df3+df4)-df5')
np.allclose(result1,result2)

True

In [11]:
#비교 연산자 pd.eval()은 연쇄 표현식을 모든 비교 연산자를 지원
result1=(df1<df2) &(df2<=df3) &(df3 != df4)
result2=pd.eval('df1<df2<=df3!=df4')
np.allclose(result1,result2)

True

In [12]:
#비트 단위 연산자 pd.eval()은 &와 | 비트 단위 연산자를 지원
result1=(df1<0.5)&(df2<0.5)|(df3<df4)
result2=pd.eval('(df1<0.5)&(df2<0.5)|(df3<df4)')
np.allclose(result1,result2)

True

In [13]:
#그 밖에 부울 표현식에서 리터럴 and 와 or 사용을 지원
result3=pd.eval('(df1<0.5)and(df2<0.5)or(df3<df4)')
np.allclose(result1,result3)

True

In [14]:
#객체 속성과 인덱스 pd.eval()은 obj.attr구문을 통해 객체 속성에 접근하는 것을 지원하고 obj[index]구문을 통해 인덱스에 접극하는 것 지원
result1=df2.T[0]+df3.iloc[1]
result2=pd.eval('df2.T[0]+df3.iloc[1]')
np.allclose(result1,result2)

True

# 열 단위의 연산을 위함 DataFrame.eval()

In [15]:
#eval()메서드의 이점은 열을 이름으로 부를 수 있다
df=pd.DataFrame(rng.rand(1000,3),columns=['A','B','C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [16]:
#pd.eval()을 사용하면 다음과 같이 세 개의 열이 있는 표현식을 계산할 수 있다
result1=(df['A']+df['B'])/(df['C']-1)
result2=pd.eval("(df.A+df.B)/(df.C-1)")
np.allclose(result1,result2)

True

In [17]:
#DataFrame.eval() 메서드를 사용하면 열을 사용하는 표현식을 훨씬 더 간결학 평가할 수 있다
result3=df.eval('(A+B)/(C-1)')
np.allclose(result1,result3)

True

In [18]:
#DataFrame.eval()에서의 할당
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


In [19]:
#새로운 열 'D'를 생성하고 거기에 다른 열로부터 계산된 값을 할당하는 데 df.eval()을 사용할 수 있다
df.eval('D=(A+B)/C',inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


In [20]:
#같은 방식으로 어느 기존 열이든 수정 가능
df.eval('D=(A-B)/C',inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


In [21]:
#DataFrame.eval()의 지역 변수
#@ 기호는 열 이름이 아닌 변수 이름을 표시해서 두 개의 네임스페이스 ,
#즉 열의 네임스페이스와 파이썬 네임스페이스를 포함하는 표현식을 효율적으로 평가할 수 있게 해준다
#DataFrame.eval()메서드에만 지원, pandas.eval()의 경우 하나의 네임스페이스에만 접근할 수 있기 때문이다
column_mean=df.mean(1)
result1=df['A']+column_mean
result2=df.eval('A+@column_mean')
np.allclose(result1,result2)

True

# DataFrame.query() 메서드

In [22]:
result1=df[(df.A<0.5) & (df.B<0.5)]
result2=pd.eval('df[(df.A<0.5) & (df.B<0.5)]')
np.allclose(result1,result2)

True

In [23]:
#필터링 연산에서는 query() 메서드를 사용할 수 있다
result2=df.query('A<0.5 and B<0.5')
np.allclose(result1,result2)

True

In [24]:
Cmean=df['C'].mean()
result1=df[(df.A<Cmean)&(df.B<Cmean)]
result2=df.query('A<@Cmean and B<@Cmean')
np.allclose(result1,result2)

True

# 성능: 이 함수를 사용해야 하는 경우

In [25]:
x=df[(df.A<0.5)&(df.B<0.5)]

In [26]:
#임시 DataFrame의 크기가 사용 가능한 시스템 메모리에 비해 상당히 크다면  eval()이나 query() 표현식을 사용하는 것이 좋다
tmp1=df.A<0.5
tmp2=df.B<0.5
tmp3=tmp1 & tmp2
x=df[tmp3]

In [27]:
#배열의 대략적인 크기를 바이트 단위로 확인할 수 있다
df.values.nbytes

32000